In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import statsmodels.api as sm
#
from multiprocessing import cpu_count
from joblib import Parallel
from joblib import delayed

import tools
import importlib
importlib.reload(tools)

/home/ec2-user/anaconda3/envs/slacktrading/lib/python3.7/site-packages/mpl_finance.py:22: DeprecationWarning: 



    Please use `mplfinance` instead (no hyphen, no underscore).

    To install: `pip install --upgrade mplfinance` 

   For more information, see: https://pypi.org/project/mplfinance/


  category=DeprecationWarning)


<module 'tools' from '/home/ec2-user/slack-trading/tools.py'>

### Download data from AWS S3

In [2]:
s3_bucket = 'slack-trading'
local_path = '../data'
tools.download_data_from_s3(s3_bucket, local_path)

### CPU Count

In [3]:
cpu_count()

16

### Begin

In [4]:
# dataset = pd.read_csv('../data/all_stock_close.csv')
dataset = pd.read_csv('../data/clean_10yr_stock_close.csv')
dataset['Date'] = pd.to_datetime(dataset['Date'])
dataset.set_index('Date', drop=True, inplace=True)
dataset.head(3)

,0001.HK,0002.HK,0003.HK,0004.HK,0005.HK,0006.HK,0007.HK,0008.HK,0010.HK,0011.HK,...,3918.HK,3933.HK,3968.HK,3988.HK,3989.HK,3993.HK,3998.HK,3999.HK,6823.HK,^HSI
Date,,,,,,,,,,,,,,,,,,,,,
2011-01-05,61.286934,42.431778,5.844519,8.506300,51.181801,27.308163,0.72,1.894841,41.338200,85.341171,...,1.048637,13.528478,14.051094,2.338585,0.43869,1.197547,2.198230,1.417399,0.266144,23757.820312
2011-01-06,62.620289,42.630985,5.762295,8.540971,51.461823,27.253111,0.73,1.922302,41.613262,85.276558,...,1.109961,13.077461,14.051094,2.344221,0.43869,1.177006,2.151601,1.408255,0.268354,23786.300781
2011-01-07,62.620289,42.630985,5.762295,8.540971,51.461823,27.253111,0.73,1.922302,41.613262,85.276558,...,1.109961,13.077461,14.051094,2.344221,0.43869,1.177006,2.151601,1.408255,0.286030,23686.630859


In [5]:
train_start = '2011-01-01'
train_end = '2011-12-31'
dataset = dataset[train_start:train_end]

In [6]:
prevday_returns = np.log(dataset.shift(1)/dataset.shift(2))
today_returns = np.log(dataset/dataset.shift(1))
#
stocks_prev = prevday_returns.columns.to_list()
stocks_today = today_returns.columns.to_list()
#
new_col = ['{}.prev'.format(c) for c in stocks_prev]
prevday_returns.columns = new_col
#
comb_df = pd.concat([prevday_returns, today_returns], axis=1)

### Fina the NaN in comb_df

In [7]:
comb_df.dropna(axis=0, how='any', inplace=True)

In [8]:
comb_df.tail(3)

,0001.HK.prev,0002.HK.prev,0003.HK.prev,0004.HK.prev,0005.HK.prev,0006.HK.prev,0007.HK.prev,0008.HK.prev,0010.HK.prev,0011.HK.prev,...,3918.HK,3933.HK,3968.HK,3988.HK,3989.HK,3993.HK,3998.HK,3999.HK,6823.HK,^HSI
Date,,,,,,,,,,,,,,,,,,,,,
2011-12-28,0.012451,0.002274,0.007808,0.011281,0.014316,0.003512,-0.029853,0.054347,0.042260,0.007547,...,0.005141,-0.002238,-0.017483,-0.020762,-0.057175,0.000000,-0.013129,-0.007435,0.000000,-0.005949
2011-12-29,0.004829,-0.007599,-0.001112,-0.025573,-0.010084,0.006987,-0.015268,-0.003839,-0.005764,-0.004845,...,0.005115,-0.009009,-0.008854,0.000000,-0.060644,-0.020495,-0.013304,-0.030305,0.000000,-0.006542
2011-12-30,-0.009142,-0.000763,-0.006697,-0.004321,-0.013606,-0.008741,0.000000,0.041437,0.009206,-0.003243,...,0.000000,0.011247,-0.002545,0.000000,0.459543,-0.017911,-0.018019,0.015268,0.013247,0.001980


In [9]:
comb_df.head(3)

,0001.HK.prev,0002.HK.prev,0003.HK.prev,0004.HK.prev,0005.HK.prev,0006.HK.prev,0007.HK.prev,0008.HK.prev,0010.HK.prev,0011.HK.prev,...,3918.HK,3933.HK,3968.HK,3988.HK,3989.HK,3993.HK,3998.HK,3999.HK,6823.HK,^HSI
Date,,,,,,,,,,,,,,,,,,,,,
2011-01-07,0.021523,0.004684,-0.014168,0.004068,0.005456,-0.002018,0.013793,0.014388,0.006632,-0.000757,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.063791,-0.004199
2011-01-10,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,-0.022347,-0.032346,-0.023367,-0.007238,-0.029847,-0.050939,0.015361,-0.013072,-0.044529,-0.006751
2011-01-11,-0.013012,-0.003120,-0.007714,-0.014720,-0.012166,-0.007096,0.000000,0.019803,-0.012352,-0.008368,...,-0.011364,-0.011019,0.005016,0.007238,-0.015264,0.015419,-0.006116,0.013072,0.044335,0.009858


In [10]:
prevday_returns = comb_df[new_col]
today_returns = comb_df[stocks_today]

### Load the regression result to identify the 1st independent variable

In [11]:
result_df = pd.read_csv('../data/2indp_var_10yr_regression_pair.csv', index_col=0)
# result_df = pd.read_csv('../data/10yr_regression_pair.csv', index_col=0)
sorted_df = result_df.sort_values(by='RSquared', ascending=False)
sorted_df.head(10)

,Prev,Today,Params,TValues,RSquared
309269,"['0266.HK.prev', '0916.HK.prev']",0952.HK,2.615029,12.118144,0.406636
309069,"['0266.HK.prev', '0406.HK.prev']",0952.HK,2.524455,11.461528,0.401171
309186,"['0266.HK.prev', '0701.HK.prev']",0952.HK,2.666816,12.304459,0.400541
309467,"['0266.HK.prev', '2302.HK.prev']",0952.HK,2.636831,12.154519,0.399246
309226,"['0266.HK.prev', '0823.HK.prev']",0952.HK,2.637518,12.144296,0.397909
309337,"['0266.HK.prev', '1104.HK.prev']",0952.HK,2.596040,11.906590,0.397453
309329,"['0266.HK.prev', '1085.HK.prev']",0952.HK,2.519347,11.337292,0.396829
309033,"['0266.HK.prev', '0335.HK.prev']",0952.HK,2.648093,12.178767,0.396276
308971,"['0266.HK.prev', '0219.HK.prev']",0952.HK,2.622200,12.040138,0.395618
309110,"['0266.HK.prev', '0529.HK.prev']",0952.HK,2.659302,12.217396,0.395152


In [12]:
sorted_df[sorted_df['RSquared'].isna()]

,Prev,Today,Params,TValues,RSquared
282580,"['0001.HK.prev', '0002.HK.prev']",0865.HK,0.0,NaN,NaN
282581,"['0001.HK.prev', '0003.HK.prev']",0865.HK,0.0,NaN,NaN
282582,"['0001.HK.prev', '0004.HK.prev']",0865.HK,0.0,NaN,NaN
282583,"['0001.HK.prev', '0005.HK.prev']",0865.HK,0.0,NaN,NaN
282584,"['0001.HK.prev', '0006.HK.prev']",0865.HK,0.0,NaN,NaN
...,...,...,...,...,...
363515,"['0001.HK.prev', '3993.HK.prev']",1159.HK,0.0,NaN,NaN
363516,"['0001.HK.prev', '3998.HK.prev']",1159.HK,0.0,NaN,NaN
363517,"['0001.HK.prev', '3999.HK.prev']",1159.HK,0.0,NaN,NaN
363518,"['0001.HK.prev', '6823.HK.prev']",1159.HK,0.0,NaN,NaN


In [13]:
sorted_df.fillna(0, inplace=True)

##### Find the row grouped by 'Today' with max RSquared within each group

In [14]:
best_prev_sofar_df = sorted_df.loc[sorted_df.groupby('Today')['RSquared'].idxmax()]
best_prev_sofar_df

,Prev,Today,Params,TValues,RSquared
658,"['0910.HK.prev', '2868.HK.prev']",0001.HK,-0.089022,3.920806,0.087150
1288,"['0868.HK.prev', '1813.HK.prev']",0002.HK,-0.101702,5.982480,0.134479
1938,"['0868.HK.prev', '1185.HK.prev']",0003.HK,-0.095374,4.656453,0.095390
2343,"['1385.HK.prev', '0391.HK.prev']",0004.HK,0.173386,3.479817,0.104310
3217,"['0910.HK.prev', '0825.HK.prev']",0005.HK,-0.088839,4.199579,0.109424
...,...,...,...,...,...
502387,"['0880.HK.prev', '0910.HK.prev']",3993.HK,0.269624,5.399720,0.137351
503325,"['1966.HK.prev', '2389.HK.prev']",3998.HK,0.426845,3.226174,0.066644
504008,"['1881.HK.prev', '2302.HK.prev']",3999.HK,0.315344,3.829024,0.086494
504325,"['0237.HK.prev', '0420.HK.prev']",6823.HK,4.106890,9.320299,0.294847


In [15]:
best_prev_sofar_df[best_prev_sofar_df['Today']=='0005.HK']

,Prev,Today,Params,TValues,RSquared
3217,"['0910.HK.prev', '0825.HK.prev']",0005.HK,-0.088839,4.199579,0.109424


### Regression

In [16]:
def get_linear_regression(pair, prevday_returns, today_returns):
    catstocks = pair['prev']
    column = pair['today']
    X = prevday_returns[catstocks].copy()
    X = sm.add_constant(X)
    y = today_returns[column]
    model = sm.OLS(y, X).fit()
    data = {
        'Prev' : catstocks,
        'Today' : column,
        'Params' : model.params.iloc[1],
        'TValues' : abs(model.tvalues.iloc[1]),
        'RSquared' : model.rsquared
    }
    # print(data)
    return data

### Generate pair list

In [17]:
catstocks = new_col
pair_list = []
for column in today_returns.columns:
    best_prev_str = best_prev_sofar_df.loc[best_prev_sofar_df['Today']==column, 'Prev'].values[0]
    best_prev_list = best_prev_str.replace('\'','').strip('][').split(', ') # convert list representation to list
    for catstock in catstocks:
        if (column not in catstock) and (catstock not in best_prev_list):
            data = {
                'prev' : best_prev_list + [catstock],
                'today' : column
            }
            pair_list.append(data)
#

### NON-Parallel processing

In [ ]:
%%time
results = [get_linear_regression(pair, prevday_returns, today_returns) for pair in pair_list]
result_df = pd.DataFrame(results)
result_df

### Parrallel processing

In [18]:
%%time
cpu = cpu_count()
print('cpu count = {}'.format(cpu))
executor = Parallel(n_jobs=cpu, backend='multiprocessing')
tasks = (delayed(get_linear_regression)(pair, prevday_returns, today_returns) for pair in pair_list)
results = executor(tasks)
result_df = pd.DataFrame(results)
result_df

cpu count = 16


/home/ec2-user/anaconda3/envs/slacktrading/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2542: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/home/ec2-user/anaconda3/envs/slacktrading/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2542: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/home/ec2-user/anaconda3/envs/slacktrading/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2542: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/home/ec2-user/anaconda3/envs/slacktrading/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2542: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwar

/home/ec2-user/anaconda3/envs/slacktrading/lib/python3.7/site-packages/statsmodels/regression/linear_model.py:1636: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
/home/ec2-user/anaconda3/envs/slacktrading/lib/python3.7/site-packages/statsmodels/regression/linear_model.py:1636: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
/home/ec2-user/anaconda3/envs/slacktrading/lib/python3.7/site-packages/statsmodels/regression/linear_model.py:1636: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
/home/ec2-user/anaconda3/envs/slacktrading/lib/python3.7/site-packages/statsmodels/regression/linear_model.py:1636: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
/home/ec2-user/anaconda3/envs/slacktrading/lib/python3.7/site-packages/statsmodels/regression/linear_model.py:1636: RuntimeWarning: inva

CPU times: user 2min 5s, sys: 5.69 s, total: 2min 11s
Wall time: 24min 21s


,Prev,Today,Params,TValues,RSquared
0,"[0910.HK.prev, 2868.HK.prev, 0002.HK.prev]",0001.HK,-0.087806,3.829764,0.087831
1,"[0910.HK.prev, 2868.HK.prev, 0003.HK.prev]",0001.HK,-0.086595,3.785724,0.090246
2,"[0910.HK.prev, 2868.HK.prev, 0004.HK.prev]",0001.HK,-0.086233,3.715045,0.088517
3,"[0910.HK.prev, 2868.HK.prev, 0005.HK.prev]",0001.HK,-0.083720,3.659267,0.096724
4,"[0910.HK.prev, 2868.HK.prev, 0006.HK.prev]",0001.HK,-0.088539,3.892441,0.088684
...,...,...,...,...,...
504803,"[0910.HK.prev, 0825.HK.prev, 3989.HK.prev]",^HSI,-0.067179,3.610343,0.092556
504804,"[0910.HK.prev, 0825.HK.prev, 3993.HK.prev]",^HSI,-0.068111,3.604902,0.086756
504805,"[0910.HK.prev, 0825.HK.prev, 3998.HK.prev]",^HSI,-0.069286,3.725171,0.086316
504806,"[0910.HK.prev, 0825.HK.prev, 3999.HK.prev]",^HSI,-0.067852,3.635410,0.088695


In [19]:
result_df.to_csv('../data/3indp_var_10yr_regression_pair.csv')

### Read Regression Result

In [20]:
result2_df = pd.read_csv('../data/3indp_var_10yr_regression_pair.csv', index_col=0)

In [21]:
sorted2_df = result2_df.sort_values(by='RSquared', ascending=False)
sorted2_df.head(10)

,Prev,Today,Params,TValues,RSquared
308432,"['0266.HK.prev', '0916.HK.prev', '0020.HK.prev']",0952.HK,2.531926,11.895655,0.433095
308598,"['0266.HK.prev', '0916.HK.prev', '0335.HK.prev']",0952.HK,2.609979,12.329334,0.431431
309113,"['0266.HK.prev', '0916.HK.prev', '3900.HK.prev']",0952.HK,2.545728,11.963035,0.431336
308525,"['0266.HK.prev', '0916.HK.prev', '0202.HK.prev']",0952.HK,2.414571,10.911227,0.430340
308751,"['0266.HK.prev', '0916.HK.prev', '0701.HK.prev']",0952.HK,2.633010,12.372730,0.425889
308617,"['0266.HK.prev', '0916.HK.prev', '0371.HK.prev']",0952.HK,2.617618,12.303363,0.425661
308791,"['0266.HK.prev', '0916.HK.prev', '0823.HK.prev']",0952.HK,2.605107,12.236845,0.425090
309030,"['0266.HK.prev', '0916.HK.prev', '2300.HK.prev']",0952.HK,2.617172,12.289345,0.424543
308634,"['0266.HK.prev', '0916.HK.prev', '0406.HK.prev']",0952.HK,2.510716,11.600564,0.424456
308500,"['0266.HK.prev', '0916.HK.prev', '0152.HK.prev']",0952.HK,2.535633,11.793893,0.424420


In [22]:
result_df = pd.read_csv('../data/2indp_var_10yr_regression_pair.csv', index_col=0)
sorted_df = result_df.sort_values(by='RSquared', ascending=False)
sorted_df.head(10)

,Prev,Today,Params,TValues,RSquared
309269,"['0266.HK.prev', '0916.HK.prev']",0952.HK,2.615029,12.118144,0.406636
309069,"['0266.HK.prev', '0406.HK.prev']",0952.HK,2.524455,11.461528,0.401171
309186,"['0266.HK.prev', '0701.HK.prev']",0952.HK,2.666816,12.304459,0.400541
309467,"['0266.HK.prev', '2302.HK.prev']",0952.HK,2.636831,12.154519,0.399246
309226,"['0266.HK.prev', '0823.HK.prev']",0952.HK,2.637518,12.144296,0.397909
309337,"['0266.HK.prev', '1104.HK.prev']",0952.HK,2.596040,11.906590,0.397453
309329,"['0266.HK.prev', '1085.HK.prev']",0952.HK,2.519347,11.337292,0.396829
309033,"['0266.HK.prev', '0335.HK.prev']",0952.HK,2.648093,12.178767,0.396276
308971,"['0266.HK.prev', '0219.HK.prev']",0952.HK,2.622200,12.040138,0.395618
309110,"['0266.HK.prev', '0529.HK.prev']",0952.HK,2.659302,12.217396,0.395152


### Upload to AWS S3

In [23]:
s3_bucket = 'slack-trading'
local_path = '../data'
tools.upload_data_to_s3(s3_bucket, local_path)

### Verify Linear regression

    catstock = pair['prev']
    column = pair['today']
    X = prevday_returns[[catstock]].copy()
    X = sm.add_constant(X)
    y = today_returns[[column]]
    model = sm.OLS(y, X).fit()
    data = {
        'Prev' : catstock,
        'Today' : column,
        'Params' : model.params.iloc[1],
        'TValues' : abs(model.tvalues.iloc[1]),
        'RSquared' : model.rsquared
    }

In [ ]:
stock1 = ['0266.HK.prev', '0916.HK.prev']
stock2 = '0952.HK'

In [ ]:
prevday_returns[stock1]

In [ ]:
today_returns[stock2]

In [ ]:
X = prevday_returns[stock1].copy()
X = sm.add_constant(X)
y = today_returns[stock2]
model = sm.OLS(y, X).fit()
print('Params {:.4f} TValues {:.4f} RSquared {:.4f}'.format(model.params.iloc[1], model.tvalues.iloc[1], model.rsquared))